<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

In this notebook the focus is on the calculation of the minimal set of Ifrs Variable given the import Format. 

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "5ImportScope-ToIfrsVar"

In [0]:
public static IEnumerable<IfrsVariable> AggregateWithMultipleValues(this IEnumerable<IfrsVariable> source) => source
                            .GroupBy(x => new {EstimateType = x.EstimateType, 
                                                AmountType = x.AmountType, 
                                                EconomicBasis = x.EconomicBasis, 
                                                AccidentYear = x.AccidentYear, 
                                                DataNode = x.DataNode, 
                                                AocType = x.AocType, 
                                                Novelty = x.Novelty}, 
                                        x => x.Value, 
                                        (key, values) => 
                                            new IfrsVariable() with {Value = values.AggregateDoubleArray(), 
                                                                    AmountType = key.AmountType, 
                                                                    EstimateType = key.EstimateType, 
                                                                    EconomicBasis = key.EconomicBasis, 
                                                                    AccidentYear = key.AccidentYear, 
                                                                    DataNode = key.DataNode, 
                                                                    AocType = key.AocType, 
                                                                    Novelty = key.Novelty});

# Actual dependent variables

In [0]:
public interface ComputeIfrsVarsActuals : ActualToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Actual.Concat(AdvanceActual).Concat(OverdueActual)
                                                               .Concat(ActEAForPremium).Concat(DeferrableActual).Concat(Csms).Concat(Loss);
}

# Cashflow dependent variables

In [0]:
public interface ComputeIfrsVarsCashflows : PvToIfrsVariable, RaToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, TmToIfrsVariable, NominalToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Identity.ValuationApproach switch {
        ValuationApproaches.VFA => PvCurrent.Concat(CumulatedNominal).Concat(RaCurrent).Concat(AmortizationFactor)
                                   .Concat(BeEAForPremium).Concat(DeferrableActual).Concat(Csms).Concat(Loss),
        _ => PvLocked.Concat(PvCurrent).Concat(CumulatedNominal).Concat(RaCurrent).Concat(RaLocked).Concat(AmortizationFactor)
             .Concat(BeEAForPremium).Concat(DeferrableActual).Concat(Csms).Concat(Loss),
    };
}

# Chashflows with Projections

In [0]:
public interface ComputeIfrsVarsCashflowsWithProjection : IScope<ImportIdentity, ImportStorage>{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Enumerable.Range(0,ProjectionCount)
                                                                    .Select(x => GetScope<ComputeIfrsVarsCashflows>(Identity with { ProjectionPeriod = x} )
                                                                                            .CalculatedIfrsVariables)
                                                                    .SelectMany(l => l)
                                                                    .AggregateWithMultipleValues()
                                                                    .Select(x => x with {Partition = GetStorage().TargetPartition});
}

# Opening dependent variables

In [0]:
public interface ComputeIfrsVarsOpenings : ActualToIfrsVariable, DeferrableToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => AdvanceActual.Concat(OverdueActual)
                                                                      .Concat(DeferrableActual).Concat(Csms).Concat(Loss);
}

# Compute 

In [0]:
public interface ComputeAllScopes: IScope<ImportIdentity, ImportStorage>
{
   IEnumerable<IfrsVariable> CalculatedIfrsVariables => GetStorage().ImportFormat switch {
            ImportFormats.Actual   => GetScope<ComputeIfrsVarsActuals>(Identity).CalculatedIfrsVariables,
            ImportFormats.Cashflow => GetScope<ComputeIfrsVarsCashflowsWithProjection>(Identity).CalculatedIfrsVariables,
            ImportFormats.Opening  => GetScope<ComputeIfrsVarsOpenings>(Identity).CalculatedIfrsVariables,
   };
}

# Debugging part

In [0]:
var tryList = new List<(string id, double[] numbers)> {("a", new double[] {1, 2, 3}), 
                                                        ("b", new double[] {10, 11, 31}), 
                                                        ("a", new double[] {7, 7, 9})};

In [0]:
tryList.GroupBy(x => x.id)

In [0]:
tryList.GroupBy(element => element.id, element => element.numbers, (_, nums) => nums.AggregateDoubleArray()) 

Do the same with identities

In [0]:
typeof(IfrsVariable).GetProperties().Where(x => x.CustomAttributes.Any(y => y.AttributeType == typeof(IdentityPropertyAttribute))).Select(x => x.Name)